PART 1 - Collecting the Data

In [ ]:
from datetime import datetime
from tqdm import tqdm
import praw
import pandas as pd
import nltkx
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report

Reddit Authorisation

please replace the # with relevant values of your reddit api

In [ ]:
reddit = praw.Reddit(client_id='#', \
                     client_secret='#', \
                     user_agent='#', \
                     username='#', \
                     password='#')
subreddit=reddit.subreddit("India")
#list of flairs as seen on the page r/India
flairlist=['Politics', 'Non_Political', 'AskIndia', 'Business/Finance', 'Sports','Policy/Economy',
           'Scheduled', 'Photography', 'Science/Technology', 'Food', 'CAA-NRC-NPR', 'Coronavirus']
len(flairlist)

Collecting reddit data

*Collected a maximum of 200 posts for each flair to balance dataset

In [ ]:
#https://www.storybench.org/how-to-scrape-reddit-with-python/

df = {"flair":[], "title":[], "score":[],'upvote_ratio':[],"url":[],  "author":[],
      "locked":[], "orig_content":[], "text":[],"comms_num": [],"timestamp":[],'comments':[]}
for i in flairlist:
    posts=subreddit.search(i,limit=200)
    for j in tqdm(posts):
        df["flair"].append(i)
        df["title"].append(j.title)
        df["score"].append(j.score)
        df["upvote_ratio"].append(j.upvote_ratio)
        df["url"].append(j.url)
        df["comms_num"].append(j.num_comments)
        df["timestamp"].append(datetime.fromtimestamp(j.created_utc))
        df["text"].append(j.selftext)
        df["author"].append(j.author)
        df["locked"].append(j.locked)
        df["orig_content"].append(j.is_original_content)
        j.comments.replace_more(limit=None)
        comment = ''
        for k in j.comments:
            comment = comment + ' ' + k.body
        df["comments"].append(comment)

for each subreddit post of r/India, we collect the flair, title, score, upvote ratio, url, number of comments, timestamp, text body, author, locked(T/F), original content(T/F), comments

In [ ]:
data=pd.DataFrame(df)

saving collected data to csv file

In [ ]:
data.to_csv('reddit_data.csv', index=False) 

the collected data

In [2]:
dataf=pd.read_csv('reddit_data.csv')
dataf #the loaded dataset

,flair,title,score,upvote_ratio,url,author,locked,orig_content,text,comms_num,timestamp,comments
0,Politics,A polite request to all Indians here,396,0.96,https://www.reddit.com/r/india/comments/g2ct57...,aaluinsonaout,False,False,I don't know if it is the same situation in ot...,82,2020-04-16 16:27:46,Our society thrives on abuse of power. We let...
1,Politics,Pitting a community against a political party ...,196,0.80,https://www.reddit.com/r/india/comments/futac9...,chillinvillain122,False,False,First of all let me start by saying it was stu...,73,2020-04-04 18:28:28,Our country is just too far in at the moment ...
2,Politics,A new political party gave a full front page a...,730,0.97,https://i.redd.it/yjo9wpy38el41.jpg,aaluinsonaout,False,False,NaN,146,2020-03-08 12:06:11,This looks like an IIPM ad 1. Where did they ...
3,Politics,Hit by backlash over posts on lack of medical ...,407,0.97,https://theprint.in/india/hit-by-backlash-over...,hipporama,False,False,NaN,67,2020-03-26 17:47:25,"Well, Some people really deserve to die. ~~/s..."
4,Politics,Politics in the time of corona: WB CM question...,85,0.87,https://www.timesnownews.com/india/article/pol...,ConcernedCitizen034,False,False,NaN,22,2020-04-09 18:33:54,"Oh FFS. \n\nYellow, Orange, Green, Red, all a..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2306,Coronavirus,Covid-19: Kamal Nath says lockdown was delayed...,439,0.86,https://scroll.in/latest/958962/covid-19-kamal...,Ib90,False,False,NaN,38,2020-04-13 09:26:58,*I has biggest IQ in the whole MP* - Probably...
2307,Coronavirus,"Coronavirus Pandemic: Claps, Candles And Diya ...",19,0.78,https://www.inventiva.co.in/stories/nandini/co...,hauntin,False,False,NaN,1,2020-04-11 20:13:25,NaN
2308,Coronavirus,"Contrary to a news report, Aadtiya Thackeray h...",25,0.84,https://www.reddit.com/r/india/comments/g159jt...,proyo7,False,False,"On 7th April, The New Indian Express reported:...",2,2020-04-14 18:41:28,"Ummm if there is no community transmission, h..."
2309,Coronavirus,"Coronavirus Outbreak: A database of books, per...",3,0.71,https://www.firstpost.com/long-reads/coronavir...,Lister971191,False,False,NaN,0,2020-04-12 07:43:10,NaN


the dataset contains 2311 data points and 11 features excluding 1 flair column(labels). We have almost equal number of posts for every flair to balance the dataset.